In [1]:
# !pip install opencv-python

In [2]:
def webcamTest():
    import cv2

    cap = cv2.VideoCapture(0)
    cap.set(3, 640)
    cap.set(4, 480)

    while True:
        ret, img= cap.read()
        cv2.imshow('Webcam', img)

        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [3]:
# !pip3 install ultralytics

In [4]:
from ultralytics import YOLO
import cv2
import math 
import time

# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# model
model = YOLO("yolo-Weights/yolov5n.pt")

# object classes
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

# person 클래스의 인덱스를 찾습니다.
person_index = classNames.index('person')

# 중심점의 히스토리를 저장할 리스트
center_points_history = []

while True:
    success, img = cap.read()
    results = model(img, stream=True)

    # 현재 시간
    current_time = time.time()

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # class name
            cls = int(box.cls[0])

            if cls == person_index:  # person 클래스만 확인
                # bounding box
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # 중심점 계산
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)

                # 중심점의 좌표와 현재 시간을 히스토리에 추가
                center_points_history.append(((center_x, center_y), current_time))

                # 바운딩 박스 그리기
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

                # confidence
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confidence --->", confidence)

                # object details
                org = [x1, y1]
                font = cv2.FONT_HERSHEY_SIMPLEX
                fontScale = 1
                color = (255, 0, 0)
                thickness = 2

                cv2.putText(img, str(classNames[cls]), org, font, fontScale, color, thickness)

    # 0.3초 이전의 중심점들을 히스토리에서 제거
    center_points_history = [(point, timestamp) for point, timestamp in center_points_history if current_time - timestamp <= 0.3]

    # 중심점들을 이미지에 그리기
    for idx, ((center_x, center_y), _) in enumerate(center_points_history):
        # 가장 최근의 점만 붉은색으로 그리기
        if idx == len(center_points_history) - 1:
            cv2.circle(img, (center_x, center_y), 3, (0, 0, 255), -1)  # RGB로 (0, 0, 255)는 붉은색입니다.
        else:
            cv2.circle(img, (center_x, center_y), 3, (0, 255, 0), -1)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


2023-11-07 00:10:04.834 Python[45456:2244827] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
PRO TIP 💡 Replace 'model=yolo-Weights/yolov5n.pt' with new 'model=yolo-Weights/yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

100%|██████████| 5.27M/5.27M [00:00<00:00, 11.2MB/s]

0: 480x640 (no detections), 87.8ms
Speed: 5.4ms preprocess, 87.8ms inference, 9.6ms postprocess per image at shape (1, 3, 480, 640)
2023-11-07 00:10:09.536 Python[45456:2244827] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.

Confidence ---> 0.36
Confidence ---> 0.73


0: 480x640 1 person, 70.6ms
Speed: 1.7ms preprocess, 70.6ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cat, 69.0ms
Speed: 1.2ms preprocess, 69.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)



Confidence ---> 0.83


0: 480x640 1 person, 65.6ms
Speed: 1.2ms preprocess, 65.6ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 82.7ms
Speed: 1.5ms preprocess, 82.7ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)



...

Confidence ---> 0.89


WARNING ⚠️ 'source' is missing. Using 'source=/Users/being/Library/Python/3.9/lib/python/site-packages/ultralytics/assets'.

image 1/2 /Users/being/Library/Python/3.9/lib/python/site-packages/ultralytics/assets/bus.jpg: 640x480 4 persons, 1 bus, 66.9ms
image 2/2 /Users/being/Library/Python/3.9/lib/python/site-packages/ultralytics/assets/zidane.jpg: 384x640 3 persons, 2 ties, 44.7ms
Speed: 3.0ms preprocess, 55.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)


Confidence ---> 0.84
Confidence ---> 0.8
Confidence ---> 0.76
Confidence ---> 0.31
Confidence ---> 0.71
Confidence ---> 0.66
Confidence ---> 0.29


error: OpenCV(4.8.1) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/highgui/src/window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


: 